In [2]:
import numpy as np

Шаг 1. Подготовить обучающую выборку

In [3]:
def prepare_training_set(q, n, k):
    X = np.random.rand(q, n)  # Обучающие векторы
    D = np.random.randint(0, 2, (q, k))  # Векторы желаемых значений выходов персептрона
    return X, D

Шаг 2. Генератором случайных чисел всем синаптическим весам и нейронным смещениям присваиваются некоторые малые случайные значения


In [4]:
def initialize_weights(n, k):
    w = np.random.uniform(-0.5, 0.5, (n + 1, k))  # Синаптические веса и нейронные смещения
    return w

Шаг 3. Общей ошибке Eобщ и всем значениям коррекции синаптических весов присвоить нулевое значение


In [5]:
def initialize_error_and_weight_correction(k):
    E_total = 0
    delta_w = np.zeros((n + 1, k))
    return E_total, delta_w

Шаг 4. Из обучающей выборки взять следующий по счету вектор Xm и подать его на входы персептрона


In [6]:
def get_next_input_vector(X, m):
    x = X[m]
    x = np.insert(x, 0, 1)  # Добавляем смещение x0 = 1
    return x

Шаг 5. Для каждого j-го нейрона вычислить взвешенную сумму входных сигналов netj и выходной сигнал yj на основании функции активации f


In [7]:
def calculate_output(x, w):
    net = np.dot(x, w)
    y = 1 / (1 + np.exp(-net))  # Функция активации сигмоид
    return y

Шаг 6. Вычислить ошибку E для текущего обучающего вектора и сложить полученное значение с общей ошибкой Eобщ


In [8]:
def calculate_error(y, d):
    error = d - y
    E = np.sum(error ** 2)
    return E, error

Шаг 7. Посчитать величину коррекции синаптических весов j-го нейрона и нейронных смещений и запомнить их (накопление изменений)


In [9]:
def accumulate_weight_correction(x, error, delta_w):
    delta_w += np.outer(x, error)
    return delta_w

Пример использования алгоритма


In [10]:
q = 10  # Количество обучающих векторов
n = 3  # Количество входов
k = 2  # Количество нейронов
learning_rate = 0.1  # Скорость обучения
epochs = 100  # Количество эпох
threshold = 0.01  # Порог ошибки для останова обучения

X, D = prepare_training_set(q, n, k)
w = initialize_weights(n, k)

for epoch in range(epochs):
    E_total, delta_w = initialize_error_and_weight_correction(k)

    for m in range(q):
        x = get_next_input_vector(X, m)
        y = calculate_output(x, w)

        # Шаг 6
        E, error = calculate_error(y, D[m])
        E_total += E

        # Шаг 7
        delta_w = accumulate_weight_correction(x, error, delta_w)

    # Шаг 8
    if E_total < threshold:  # Критерий останова обучения по порогу ошибки
        print("Обучение завершено досрочно после", epoch + 1, "эпох")
        break

    # Дополнительный критерий останова: проверка изменения общей ошибки
    if epoch > 0 and abs(E_total - prev_E_total) < 0.001:
        print("Обучение завершено досрочно после", epoch + 1, "эпох (ошибка стабилизировалась)")
        break
    prev_E_total = E_total

    # Шаг 9
    w += learning_rate * delta_w

print("Окончательные веса:", w)
print("Общая ошибка:", E_total)

Окончательные веса: [[ 1.51444805 -0.28576306]
 [ 0.52176032  2.72331752]
 [-2.66343465 -3.30378247]
 [-0.9698662   3.07281395]]
Общая ошибка: 2.9061610734553494


Теперь добавь реальное обучение keras данным алгоритмом

In [11]:
!pip install tensorflow
!pip install keras
import numpy as np
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

In [29]:
# Шаг 1. Подготовить обучающую выборку
def prepare_training_set(q, n, k):
    X = np.random.rand(q, n)  # Обучающие векторы
    D = np.random.randint(0, 2, (q, k))  # Векторы желаемых значений выходов персептрона
    return X, D

# Пример использования алгоритма с Keras
q = 100  # Количество обучающих векторов
n = 3  # Количество входов
k = 2  # Количество нейронов
epochs = 100  # Количество эпох
threshold = 0.1  # Порог ошибки для останова обучения

X, D = prepare_training_set(q, n, k)

# Создание модели персептрона с использованием Keras
model = Sequential()
model.add(Dense(k, input_dim=n, activation='sigmoid'))

# Компиляция модели
model.compile(loss='mean_squared_error', optimizer='sgd')

# Обучение модели
history = model.fit(X, D, epochs=epochs, verbose=0)

# Определение причины останова обучения
final_epoch = len(history.history['loss'])
if history.history['loss'][-1] < threshold:
    stop_reason = "Обучение завершено досрочно, так как ошибка стала меньше порогового значения."
else:
    stop_reason = "Обучение завершено после достижения максимального количества эпох."

# Тестирование модели и вывод результатов
predictions = model.predict(X)
print("Predictions:")
print(predictions)
print(stop_reason)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predictions:
[[0.46094176 0.5231327 ]
 [0.32582286 0.60179394]
 [0.42210585 0.4930038 ]
 [0.42615354 0.5235824 ]
 [0.39255127 0.6133817 ]
 [0.5211703  0.41276115]
 [0.37259308 0.6292958 ]
 [0.38468513 0.60787046]
 [0.3759055  0.5968068 ]
 [0.39951274 0.5619012 ]
 [0.42351064 0.49388278]
 [0.31013358 0.6508396 ]
 [0.3854356  0.60684264]
 [0.38665667 0.5111435 ]
 [0.35741168 0.6141499 ]
 [0.5050088  0.47053224]
 [0.46359944 0.4598321 ]
 [0.41852692 0.483938  ]
 [0.50198877 0.5005281 ]
 [0.42396906 0.46892342]
 [0.50063205 0.41578752]
 [0.4864701  0.41262594]
 [0.42825934 0.55055356]
 [0.3211965  0.5543542 ]
 [0.37814796 0.5313806 ]
 [0.44966373 0.418601  ]
 [0.39259592 0.48537418]
 [0.40843004 0.5639297 ]
 [0.3610813  0.49941665]
 [0.35393712 0.6012803 ]
 [0.4754112  0.3994543 ]
 [0.39792553 0.6110706 ]
 [0.39838287 0.5534105 ]
 [0.3053078  0.6509017 ]
 [0.34032375 0.6362909 ]
 [0.280388   0.62582576]
 [0.44809037 0.5035584 ]
 [0.38880658 0.49049404]